In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pylab as plt
import xgboost as xgb


%matplotlib inline
plt.style.use('ggplot')

/home/rudolph/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
train_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv").drop("Unnamed: 0", axis=1)
train_target = pd.read_csv("train_target.csv")

print("Shapes of data:", train_data.shape, test_data.shape, train_target.shape)
print("Proportion train/test: ", int(train_data.shape[0]/test_data.shape[0]))

Shapes of data: (27594, 20) (13593, 20) (27594, 1)
Proportion train/test:  2


In [ ]:
yes_indexes = train_target[train_target["1"]==1].index
no_indexes = train_target[train_target["1"]==0].index

In [ ]:
new_train_yes = train_data.iloc[yes_indexes]
new_train_no = train_data.iloc[no_indexes][:int(2*len(new_train_yes))]
new_train_data = pd.concat([new_train_no, new_train_yes], axis=0).reset_index().drop("index", axis=1)
new_train_target = pd.concat([train_target.iloc[no_indexes][:int(2*len(new_train_yes))], train_target.iloc[yes_indexes]], axis=0).reset_index().drop("index", axis=1)

### Preprocessing

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from pandas import get_dummies

In [8]:
le = LabelEncoder()
enc_train_data = np.array(pd.get_dummies(train_data.apply(le.fit_transform)))

In [9]:
train_target = np.array(train_target)
c, r = train_target.shape
train_target = train_target.reshape(c,)

### XGBoost

In [18]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve, auc


In [25]:
def xgb_best_params(data, target):
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=442)
    """skf = StratifiedKFold(target, n_folds=10, random_state=7)
    
    for train_index, test_index in skf:
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = target[train_index], target[test_index]
    """
    
    gbm = xgb.XGBClassifier(n_estimators=100, max_depth=7, max_delta_step=3)
    gbm.fit(X_train, y_train, eval_metric='auc')
    
    predictions = gbm.predict_proba(X_test)
    
    fpr, tpr, thresholds = roc_curve(y_test, predictions[:, 1], pos_label=1)
    print("AUC : %.4g" % auc(fpr, tpr))
    

In [28]:
def xgb_grid_search(data, target):
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=442)
   
    """sfk = StratifiedKFold(target, n_folds=5)
    for train_index, test_index in sfk:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        """
    
    gbm_params = {
        'n_estimators': [100, 150, 250, 350, 400],
        'learning_rate': [0.1, 0.3]
    }

    print("GridSeachCV proceeding...")
    gbm = GridSearchCV(xgb.XGBClassifier(max_depth=5, min_child_weight=1, gamma=0,
                                         subsample=0.8, colsample_bytree=8, scale_pos_weight=1), gbm_params, n_jobs=3, cv=5, verbose=2)
    gbm.fit(X_train, y_train)
    print("Done.")
    
    print(101*"="+ "\nBEST PARAMETERS: ", gbm.best_params_, "\n"+101*"="+"\n")
    predictions = gbm.predict_proba(X_test)
    
    fpr, tpr, thresholds = roc_curve(y_test, predictions[:, 1], pos_label=1)
    print("AUC : %.4g" % auc(fpr, tpr))
    

In [21]:
submission = xgb_best_params(new_train_data, new_train_target)

NameError: name 'new_train_data' is not defined

In [ ]:

from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score

model = xgboost.XGBClassifier()
kfold = StratifiedKFold(Y, n_folds=10, random_state=7)
results = cross_val_score(model, X, Y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

### Weighted SVC

In [11]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.combine import SMOTETomek

In [17]:
# 'SMOTE SVM'
ratio = 'auto'
svm_args={'class_weight': 'auto'}
svmsmote = SMOTE(ratio=ratio, kind='svm', **svm_args)
svsx, svsy = svmsmote.fit_sample(enc_train_data, train_target)

/home/rudolph/anaconda3/lib/python3.5/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


In [26]:
xgb_best_params(svsx, svsy)

AUC : 1


In [ ]:
xgb_grid_search(svsx, svsy)

GridSeachCV proceeding...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=100, learning_rate=0.1 .............................
[CV] n_estimators=100, learning_rate=0.1 .............................
[CV] n_estimators=100, learning_rate=0.1 .............................
